# $\textrm{Introdution and Background}$

# $\textrm{Data Description}$

# $\textrm{Data Cleaning/Pre-processing}$

In [1]:
import pandas as pd
import numpy as np
import scipy.interpolate as sp_interpolate

## Income

In [2]:
income = pd.read_csv('Income_Data/cities_median_income.csv')

In [3]:
income.head()

,City,Median Income 2016,Median Income 2015,Median Income 2014,Median Income 2013,Median Income 2012,Median Income 2011,Median Income 2010
0,"Birmingham, AL","46,346","44,463","44,447","44,716","44,710","45,423","45,198"
1,"Montgomery, AL","63,847","62,201","62,568","62,418","61,896","62,645","61,229"
2,"Anchorage, AK","102,500","99,630","98,317","96,935","95,632","94,073","91,251"
3,"Juneau, AK","104,005","102,355","97,588","96,025","95,433","93,994","89,373"
4,"Phoenix, AZ","69,216","66,345","65,770","65,199","65,968","66,709","66,996"


In [4]:
# rename columns
income.columns = ['city', '2016', '2015', '2014', '2013', '2012', '2011', '2010']

In [5]:
# unpivot
income = pd.melt(income, id_vars=['city'], var_name='year', value_name='median income') \
           .sort_values(['city', 'year']) \
           .reset_index(drop=True)

In [6]:
# convert to MultiIndex
income.set_index(['city', 'year'], inplace=True)

In [7]:
income.head()

median income
city       year              
Albany, NY 2010        73,470
           2011        75,821
           2012        76,707
           2013        77,687
           2014        78,285

## Unemployment

In [8]:
unemployment = pd.read_csv('Income_Data/cities_unemployment_rate.csv')

In [9]:
unemployment.head()

,City,Unemployment Rate 2016,Unemployment Rate 2015,Unemployment Rate 2014,Unemployment Rate 2013,Unemployment Rate 2012,Unemployment Rate 2011,Unemployment Rate 2010
0,"Birmingham, AL",11.8%,13.3%,14.5%,15.3%,14.5%,13.6%,12.9%
1,"Montgomery, AL",8.4%,8.8%,8.9%,8.9%,8.7%,8.6%,7.9%
2,"Anchorage, AK",6.0%,6.7%,6.9%,7.3%,7.3%,7.2%,7.4%
3,"Juneau, AK",4.9%,5.2%,5.6%,5.3%,5.1%,5.3%,5.8%
4,"Phoenix, AZ",7.7%,8.8%,9.9%,10.6%,9.7%,8.7%,7.4%


In [10]:
# rename columns
unemployment.columns = ['city', '2016', '2015', '2014', '2013', '2012', '2011', '2010']

In [11]:
# unpivot
unemployment = pd.melt(unemployment, id_vars=['city'], var_name='year', value_name='unemployment rate') \
                 .sort_values(['city', 'year']) \
                 .reset_index(drop=True)

In [12]:
# convert percentages to decimals
unemployment['unemployment rate'] = unemployment['unemployment rate'] \
                                    .map(lambda rate: float(rate.rstrip('%'))/100)

In [13]:
# convert to MultiIndex
unemployment.set_index(['city', 'year'], inplace=True)

In [14]:
unemployment.head()

unemployment rate
city       year                   
Albany, NY 2010              0.060
           2011              0.065
           2012              0.070
           2013              0.073
           2014              0.068

## Traffic

In [15]:
traffic = pd.read_csv('Income_Data/cities_mean_commute_time.csv')

In [16]:
traffic.head()

,City,Mean Commute Time 2016,Mean Commute Time 2015,Mean Commute Time 2014,Mean Commute Time 2013,Mean Commute Time 2012,Mean Commute Time 2011,Mean Commute Time 2010
0,"Birmingham, AL",21.4,21.4,21.6,21.7,21.8,21.7,22.0
1,"Montgomery, AL",20.0,19.3,19.5,19.3,19.5,19.5,19.7
2,"Anchorage, AK",19.1,19.5,19.4,19.4,19.0,18.6,18.2
3,"Juneau, AK",15.6,15.1,14.8,15.1,15.5,15.2,15.3
4,"Phoenix, AZ",25.1,24.8,24.5,24.4,24.4,24.7,24.8


In [17]:
# rename columns
traffic.columns = ['city', '2016', '2015', '2014', '2013', '2012', '2011', '2010']

In [18]:
# unpivot
traffic = pd.melt(traffic, id_vars=['city'], var_name='year', value_name='mean commute time') \
           .sort_values(['city', 'year']) \
           .reset_index(drop=True)

In [19]:
# convert to MultiIndex
traffic.set_index(['city', 'year'], inplace=True)

In [20]:
traffic.head()

mean commute time
city       year                   
Albany, NY 2010               19.7
           2011               19.9
           2012               20.0
           2013               20.0
           2014               20.1

## Crime

In [21]:
crime = pd.read_pickle('crime_data/crime_data.pkl')

In [22]:
crime.head()

arson assault burglary murder rape robbery  theft  \
city           year                                                     
Birmingham, AL 2002   211    1697     4389     65  239    1186  11640   
               2003   175    1706     4831     85  204    1352  11934   
               2004   142    1593     5156     59  240    1369  11970   
               2005   136    1675     4933    104  241    1429  11962   
               2006   228    1422     4813    104  220    1429  12113   

                    vehicle theft population  
city           year                           
Birmingham, AL 2002          2049        NaN  
               2003          2809        NaN  
               2004          2351        NaN  
               2005          2028        NaN  
               2006          2081        NaN

In [23]:
# only keep 2009–2016
crime.drop([str(year) for year in range(2000, 2008+1)], level='year', inplace=True)

In [24]:
# the only population value we're missing (2009 onward) is Chicago 2013,
# so we'll fill it with the mean of Chicago's 2012 and 2014 populations
crime.loc['Chicago, IL', '2013'].loc['population'] \
    = crime.loc['Chicago, IL'].loc[['2012', '2014'], 'population'].mean()

In [25]:
# rescale crime values to occurence per capita
for col_name in crime.columns:
    crime[col_name] = crime[col_name]/crime['population']
crime.drop('population', axis=1, inplace=True)
# append "rate" to column names
crime.rename(columns=lambda col_name: col_name + " rate", inplace=True)

In [26]:
# These cities are missing all data points for these metrics,
# so we'll fill those with zeros
crime.loc['Boston, MA', 'arson rate'] = 0
crime.loc['New York City, NY', 'arson rate'] = 0
crime.loc['Huntington, WV', 'burglary rate'] = 0

In [27]:
# These cities are missing half or more data points for these metrics,
# so we'll fill them with the means of the available data points for those metrics for each city
crime.loc['Montgomery, AL', 'arson rate'] =   crime.loc['Montgomery, AL', 'arson rate']                   \
                                              .fillna(crime.loc['Montgomery, AL', 'arson rate'].mean())   \
                                              .values
crime.loc['Tucson, AZ', 'theft rate'] =       crime.loc['Tucson, AZ', 'theft rate']                       \
                                              .fillna(crime.loc['Tucson, AZ', 'theft rate'].mean())       \
                                              .values
crime.loc['New Orleans, LA', 'arson rate'] =  crime.loc['New Orleans, LA', 'arson rate']                  \
                                              .fillna(crime.loc['New Orleans, LA', 'arson rate'].mean())  \
                                              .values
crime.loc['Philadelphia, PA', 'arson rate'] = crime.loc['Philadelphia, PA', 'arson rate']                 \
                                              .fillna(crime.loc['Philadelphia, PA', 'arson rate'].mean()) \
                                              .values

In [28]:
# These cities are missing just a few data points,
# so we'll fill them by linear interpolation
def interpolate_crime_data(series):
    x = series.dropna().index.values.astype(int) # year
    y = series.dropna().values # data
    
    f = sp_interpolate.interp1d(x, y, fill_value='extrapolate')
    x_all = series.index.values.astype(int)
    return f(x_all)

missing_data = [
    ('Birmingham, AL', 'arson rate'),
    ('Phoenix, AZ', 'arson rate'),
    ('Chicago, IL', 'arson rate'),
    ('Chicago, IL', 'assault rate'),
    ('Chicago, IL', 'rape rate'),
    ('Indianapolis, IN', 'arson rate'),
    ('Topeka, KS', 'arson rate'),
    ('Louisville, KY', 'arson rate'),
    ('Lincoln, NE', 'arson rate'),
    ('Omaha, NE', 'arson rate'),
    ('Albuquerque, NM', 'arson rate'),
    ('Santa Fe, NM', 'arson rate'),
    ('Albany, NY', 'arson rate'),
    ('Columbus, OH', 'arson rate'),
    ('Providence, RI', 'vehicle theft rate'),
    ('West Valley City, UT', 'burglary rate'),
    ('Madison, WI', 'arson rate')
]

for city, injustice in missing_data:
    crime.loc[city, injustice] = interpolate_crime_data(crime.loc[city, injustice])

In [29]:
crime.head()

arson rate assault rate burglary rate  murder rate  \
city           year                                                        
Birmingham, AL 2009  0.000593738   0.00615289     0.0220739  0.000285874   
               2011  0.000576766   0.00898442     0.0272252  0.000253214   
               2012  0.000548611   0.00954207      0.022057  0.000314162   
               2013  0.000616101   0.00774525     0.0189527  0.000297168   
               2014  0.000683591   0.00982486     0.0176791   0.00024515   

                       rape rate robbery rate theft rate vehicle theft rate  
city           year                                                          
Birmingham, AL 2009  0.000870816   0.00505777    0.05078         0.00701051  
               2011  0.000853426   0.00474074  0.0493393         0.00709469  
               2012  0.000712725   0.00460927  0.0423978         0.00488592  
               2013  0.000839619   0.00457073  0.0408536         0.00697167  
               2014  0.000858025   0.00495486  0.0412182         0.00676991

## Happiness

# $\textrm{Data Visualization}$

In [30]:
import matplotlib.pyplot as plt

# $\textrm{Data Anaylsis and Results}$

# $\textrm{Privacy/Ethics Considerations}$

# $\textrm{Conclusions and Discussion}$